# LAB: Feature Selection

En este lab vamos a explorar selección de características. Seguiremos trabajando sobre el dataset de Titanic.

Ante que nada, carguemos

- Paquetes estándar
- El dataset

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv('train.csv')
df.columns.values

Descartemos las variables que no vamos a usar

In [ ]:
X = df[[u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Fare', u'Embarked']]
y = df[u'Survived']

Chequeemos la existencia de valores perdidos:

In [ ]:
for i in X.columns.values:
    print i,":", sum(X[i].isnull())

In [ ]:
s = X['Embarked'].fillna('Q')
X.loc[:,'Embarked'] = s

Ahora generemos un pipeline que realice las siguientes tareas de preprocesamiento de la información:

- Normalize las variables numéricas
- Genere dummies de las variables categóricas

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder,Imputer, OneHotEncoder, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from IPython.core.debugger import Tracer

In [ ]:
class DataFrameColumnExtracter(TransformerMixin):

    def __init__(self, columns, y = None):
        self.columns = columns

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        col_list = []
        for c in self.columns:
            col_list.append(c)
        return (X[col_list])
        #return np.array(x)

In [ ]:
ndebug_here = Tracer()
pipeline = Pipeline([
    ('union', FeatureUnion([
        ('cont', Pipeline([
            ('extract', DataFrameColumnExtracter([u'Age',u'Fare',u'Parch',u'SibSp'])),
            ('impute', Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
            ('scale', StandardScaler())
        ])),
        ('cat', Pipeline([
            ('extract', DataFrameColumnExtracter([u'Pclass', u'Sex',u'Embarked'])),
            ('categ', LabelEncoder()),
            ('onehot', OneHotEncoder(handle_unknown='error', n_values='auto', sparse=False)),
        ])),
    ])
)])
        

## 1 Nombres de columnas

Parece que se han perdido algunos nombres de columna! Vamos a agegarlas manualmente:
- pipe_edad => 'scaled_age'
- pipe_puerto => 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S'
- pipe_sexo => 'male'
- pipe_tarifa => 'scaled_fare'

Vamos a necesitar:

1. Crear un nuevo dataframe Pandas llamado "Xt" con los nombres de columna apropiados y llenaron con los datos de "X_transf".
2. Dado que el Pipeline descarta las columnas "SibSp" y "Parch", agreguémoslos como están en el nuevo dataframe


In [ ]:
new_cols = ['scaled_age', 'Pclass_1', 'Pclass_2', 'Pclass_3',
            'Embarked_C', 'Embarked_Q', 'Embarked_S',
            'male', 'scaled_fare']

Xt = pd.DataFrame(X_transf, columns=new_cols)
Xt = pd.concat([Xt, X[[u'SibSp', u'Parch']]], axis = 1)
Xt.head()

## 2. Feature selection

Utilicemos el método "SelectKBest" de scikit learn a ver cuáles son las top 5 características.

Cuáles son?


Guardémoslas en una variable llamada "kbest_columns"

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
selector = SelectKBest(f_classif, k=5)
selected_data = selector.fit_transform(Xt, y)
kbest_columns = Xt.columns[selector.get_support()]
Xtbest = pd.DataFrame(selected_data, columns=kbest_columns)
Xtbest.head()

## 3. Eliminación recursiva de características

En Scikit Learn también vamos a encontrar una clase para realizar una eliminación recursiva de características. La misma se llama "RFECV". Usémosla en combinación de un modelo de regresión logística para ver qué características serán conservadas con este método.

Guardémoslas en una variable llamada "rfecv_columns"

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

In [ ]:
estimator = LogisticRegression()
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(Xt, y)
rfecv_columns = Xt.columns[selector.support_]
rfecv_columns

## 4. Coeficientes de Regresión Logística

Veamos si los coeficientes de una RL se condicen.

- Creá un modelo de regresión logística
- Ejecutá un grid search sobre los parámetros "penalty type" y "C strength" para encontrar la mejor combinación
- Ordená los coeficientes obtenidos por valor absoluto (módulo). El top 5 coincide con los de arriba? Por qué/Por qué no? (Pista: Están todos los valores en la misma escala?)

Guardemos las que querramos mantener en una variable llamada "lr_columns"

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
model = GridSearchCV(LogisticRegression(), {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                                            'penalty': ['l1', 'l2']})
model.fit(Xt, y)

In [ ]:
model.best_estimator_

In [ ]:
model.best_score_

In [ ]:
coeffs = pd.DataFrame(model.best_estimator_.coef_, columns = Xt.columns)
coeffs_t = coeffs.transpose()
coeffs_t.columns = ['Surv coeff']
coeffs_t.abs().sort_values('Surv coeff', ascending=False)

In [ ]:
# keep the ones with coeff above 0.3
lr_columns = coeffs.columns[(coeffs.abs() > 0.3).values[0]]

## 5. Comparar sets de características

Usá el "best estimator" del punto anterior sobre los sets de características obtenidos:

- "kbest_columns"
- "rfecv_columns"
- "lr_columns"
- "all_columns"

Usá validación cruzada (cross_val_score) para evaluar los modelos 
Preguntas:

- Cuál obtuvo mejores resultados?
- Hay diferencias signigicativas?
- Cuál es la mejor opción? Por qué?

In [ ]:
from sklearn.cross_validation import cross_val_score

def score(X):
    scores = cross_val_score(model.best_estimator_, X, y)
    return scores.mean(), scores.std()

all_scores = [
    score(Xt[kbest_columns]),
    score(Xt[rfecv_columns]),
    score(Xt[lr_columns]),
    score(Xt)]

pd.DataFrame(all_scores, columns=['mean score', 'std score'], index = ['kbest', 'rfecv', 'lr', 'all'])


## Bonus

Creá un grágico de barras para mostrar los coeficientes de la regresión logística.

In [ ]:
coeffs_t.sort_values('Surv coeff').plot(kind='bar')